In [1]:
import pandas as pd

# Replace 'file_path.tsv' with the actual path to your TSV file
file_path = 'raw_data/1718053372032.metadata.tsv'

# Read the TSV file
rawdata = pd.read_csv(file_path, sep='\t')

# Display the first few rows of the dataframe
print(rawdata.head())


                               strain            virus    gisaid_epi_isl  \
0  hCoV-19/Australia/QLD0x0140B0/2024  betacoronavirus  EPI_ISL_18979667   
1  hCoV-19/Australia/QLD0x01418D/2024  betacoronavirus  EPI_ISL_19009760   
2  hCoV-19/Australia/QLD0x014092/2024  betacoronavirus  EPI_ISL_18979706   
3  hCoV-19/Australia/QLD0x01410E/2024  betacoronavirus  EPI_ISL_18979645   
4    hCoV-19/Australia/TAS006878/2024  betacoronavirus  EPI_ISL_18967246   

  genbank_accession        date   region    country    division location  \
0                 ?  2024-02-08  Oceania  Australia  Queensland      NaN   
1                 ?  2024-03-12  Oceania  Australia  Queensland      NaN   
2                 ?  2024-02-05  Oceania  Australia  Queensland      NaN   
3                 ?  2024-02-05  Oceania  Australia  Queensland      NaN   
4                 ?  2024-02-01  Oceania  Australia    Tasmania      NaN   

  region_exposure  ... pangolin_lineage GISAID_clade  \
0         Oceania  ...        

In [3]:
distinct_values = rawdata['pangolin_lineage'].unique()
distinct_values

array(['JN.1.35', 'JN.1.32', 'JN.1.4', 'JN.1', 'JN.1.5', 'KW.1',
       'XBC.1.3', 'JN.2', 'JN.1.1', 'JN.1.32.1', 'XDQ', 'JN.1.20',
       'JN.1.4.5', 'JN.1.11.1', 'JN.1.8', 'JN.1.22', 'JN.1.3',
       'JN.1.28.1', 'KW.1.1', 'JN.1.29', 'JN.1.30.1', 'JN.1.18',
       'JN.1.19', 'JN.1.4.7', 'JN.1.39', 'KW.1.2', 'JN.1.16', 'BA.2.86',
       'JN.1.1.6', 'JN.18', 'JN.1.6', 'KV.1', 'JN.1.4.6', 'JN.14',
       'JN.1.28', 'JN.1.51', 'JN.1.56', 'JN.1.11', 'JN.1.31', 'LE.2',
       'KP.1', 'HK.3.2', 'HK.3', 'XDD', 'JG.3', 'KP.2', 'JN.1.16.2',
       'XBB.1.16.6', 'JN.2.1', 'JD.1.1', 'XDK.1', 'LJ.1', 'FL.1.5.2',
       'JN.1.48', 'JN.1.51.1', 'JN.1.47', 'XDK', 'JN.1.52', 'BA.2.86.1',
       'JN.1.43', 'JN.1.17', 'JN.1.10', 'EG.5.1.8', 'HV.1', 'JE.1.1',
       'HK.1', 'KP.4.1', 'JN.1.7', 'JN.1.8.1', 'KU.1', 'JN.1.37',
       'JN.1.9', 'BA.2.86.3', 'KU.2', 'XDR', 'JN.1.36', 'JN.1.45',
       'JN.1.4.3', 'JN.1.43.1', 'JN.1.23', 'KV.2', 'EG.5.1.1', 'JN.1.34',
       'JN.1.4.1', 'JN.1.49', 'EG.5.1.6',

In [27]:
data = rawdata

def covid_tests_data_cleaning(data):
    # Filter
    data = data[(data['region'] == 'Oceania') & (data['country'] == 'Australia')]
    data = data[data['host'] == 'Human']
    data[['age', 'sex']] = data[['age', 'sex']].replace('unknown', pd.NA)
    
    # Renaming columns
    data.rename(columns={'pangolin_lineage': 'lineage'}, inplace=True)
    data.rename(columns={'gisaid_epi_isl': 'id'}, inplace=True)

    # List of columns to keep
    columns_to_keep = [
        # virus
        'lineage', 'id', 'date', 
        # location
        'division', 'location', 
        # exposure
        'region_exposure', 'country_exposure', 'division_exposure',
        # patient 
        'age', 'sex', 
        # submission
        'originating_lab', 'submitting_lab', 'date_submitted'
    ]

    # Keep only the specified columns
    data = data.loc[:, columns_to_keep]
    
    # Function to extract substring before the second '.'
    def extract_lineage_prefix(s):
        parts = s.split('.')
        if len(parts) > 2:
            return '.'.join(parts[:2])
        return s

    # Apply the function to the pangolin_lineage column
    data['strain'] = data['lineage'].apply(extract_lineage_prefix)
    
    def extract_id(s):
        return s.split('_')[-1]

    # Apply the function to the gisaid_epi_isl column
    data['id'] = data['id'].apply(extract_id)

    # Change data type
    
    # Convert to datetime using the specified formats
    def parse_dates(date_series):
        date_formats = ["%d/%m/%Y", "%Y-%m-%d"]
        for fmt in date_formats:
            parsed_dates = pd.to_datetime(date_series, format=fmt, errors='coerce')
            if parsed_dates.notna().all():
                return parsed_dates
        return pd.to_datetime(date_series, errors='coerce') 
    
    data['date'] = parse_dates(data['date'])
    data = data.dropna(subset=['date'])
    data['date_submitted'] = parse_dates(data['date_submitted'])
    
    data['age'] = pd.to_numeric(data['age'], errors='coerce')


    return data

data = covid_tests_data_cleaning(data)

# Display the first few rows of the filtered dataframe
print(data.head())

   lineage        id       date    division location region_exposure  \
0  JN.1.35  18979667 2024-02-08  Queensland      NaN         Oceania   
1  JN.1.32  19009760 2024-03-12  Queensland      NaN         Oceania   
2   JN.1.4  18979706 2024-02-05  Queensland      NaN         Oceania   
3     JN.1  18979645 2024-02-05  Queensland      NaN         Oceania   
4   JN.1.5  18967246 2024-02-01    Tasmania      NaN         Oceania   

  country_exposure division_exposure  age   sex  \
0        Australia        Queensland  NaN  <NA>   
1        Australia        Queensland  NaN  <NA>   
2        Australia        Queensland  NaN  <NA>   
3        Australia        Queensland  NaN  <NA>   
4        Australia          Tasmania  NaN  <NA>   

                                     originating_lab  \
0  Public Health Virology, Forensic and Scientifi...   
1  Public Health Virology, Forensic and Scientifi...   
2                      Sullivan Nicolaides Pathology   
3                   Forensic and Sci

In [14]:
data['id'].is_unique

True

In [15]:
data['strain'].unique()

array(['JN.1', 'KW.1', 'XBC.1', 'JN.2', 'XDQ', 'BA.2', 'JN.18', 'KV.1',
       'JN.14', 'LE.2', 'KP.1', 'HK.3', 'XDD', 'JG.3', 'KP.2', 'XBB.1',
       'JD.1', 'XDK.1', 'LJ.1', 'FL.1', 'XDK', 'EG.5', 'HV.1', 'JE.1',
       'HK.1', 'KP.4', 'KU.1', 'KU.2', 'XDR', 'KV.2', 'LA.1', 'KP.3',
       'KZ.1', 'KS.1', 'XDV.1', 'XDQ.1', 'XCH.1', 'XDN', 'FY.5', 'KR.1',
       'LA.2', 'LB.1', 'JQ.2', 'XDP.1', 'XDP', 'LF.1', 'LG.1'],
      dtype=object)

In [28]:
# Read All tsv files
import os

# Define the directory containing the TSV files
dir_path = 'raw_data'

# Initialize an empty list to hold the DataFrames
dataframes = []

# Loop through all files in the directory
for file_name in os.listdir(dir_path):
    # Check if the file is a TSV file
    if file_name.endswith('.tsv'):
        # Read the TSV file into a DataFrame
        file_path = os.path.join(dir_path, file_name)
        df = pd.read_csv(file_path, sep='\t')
        # Append the DataFrame to the list
        try:
            dataframes.append(covid_tests_data_cleaning(df))
        except Exception as e:
            print(f"Error in loading {file_name}\n{e}")

data = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the filtered dataframe
print(data.head())

     lineage       id       date                      division location  \
0  AY.39.1.3  4636733 2021-09-28  Australian Capital Territory      NaN   
1  AY.39.1.2  4636712 2021-09-27  Australian Capital Territory      NaN   
2    AY.39.1  4552404 2021-09-13               New South Wales   Sydney   
3  AY.39.1.2  4636623 2021-09-23  Australian Capital Territory      NaN   
4  AY.39.1.2  4636635 2021-09-23  Australian Capital Territory      NaN   

  region_exposure country_exposure             division_exposure  age   sex  \
0         Oceania        Australia  Australian Capital Territory  NaN  <NA>   
1         Oceania        Australia  Australian Capital Territory  NaN  <NA>   
2         Oceania        Australia               New South Wales  NaN  <NA>   
3         Oceania        Australia  Australian Capital Territory  NaN  <NA>   
4         Oceania        Australia  Australian Capital Territory  NaN  <NA>   

                                     originating_lab  \
0                 

In [29]:
print(data.shape)

(233848, 14)


In [30]:
# Assuming 'data' is your DataFrame
na_counts = data.isna().sum()

print(na_counts)

lineage                   0
id                        0
date                      0
division                  7
location             167273
region_exposure           0
country_exposure          0
division_exposure         7
age                  155677
sex                  143008
originating_lab         128
submitting_lab            0
date_submitted            0
strain                    0
dtype: int64


In [31]:

# Check if 'gisaid_epi_isl' is a unique key
is_unique_key = data['id'].nunique() == len(data)

if is_unique_key:
    print("'gisaid_epi_isl' is a unique key.")
else:
    print("'gisaid_epi_isl' is not a unique key.")

'gisaid_epi_isl' is a unique key.


In [32]:

# Display the first few rows of the dataframe
print(*data.columns, sep='\n')


lineage
id
date
division
location
region_exposure
country_exposure
division_exposure
age
sex
originating_lab
submitting_lab
date_submitted
strain
